In [ ]:
# Import Snowpark and session
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, coalesce, row_number
from snowflake.snowpark.window import Window

# Define session parameters
session_params = {
    "account": "<YOUR_ACCOUNT>",
    "user": "<YOUR_USERNAME>",
    "password": "<YOUR_PASSWORD>",
    "role": "<YOUR_ROLE>",
    "warehouse": "<YOUR_WAREHOUSE>",
    "database": "COMMODITIES",
    "schema": "CUSTOMERS"
}

# Create a session
session = Session.builder.configs(session_params).create()

In [ ]:
# Step 1: Define the VesselListHL query
last_positions = session.table("LAST_POSITIONS")

# Filter for specific Hapag Lloyd vessels
vessel_names = [
            'BERLIN EXPRESS', 'MANILA EXPRESS', 'HANOI EXPRESS', 'BUSAN EXPRESS', 
            'SINGAPORE EXPRESS', 'DAMIETTA EXPRESS', 'HAMBURG EXPRESS', 
            'AL DAHNA', 'AL MURAYKH', 'AL NEFUD', 'AL ZUBARA', 
            'BARZAN', 'TIHAMA', 'AFIF', 'AL DHAIL', 'AL JASRAH', 
            'AL JMELIYAH', 'AL MASHRAB', 'AL MURABBA', 'AL NASRIYAH', 
            'LINAH', 'BRUSSELS EXPRESS', 'SALAHUDDIN', 'UMM QARN', 
            'AIN SNAN EXPRESS', 'ALULA', 'AL QIBLA', 'AL RIFFA', 
            'JEBEL ALI', 'TAYMA', 'TANGIER EXPRESS', 'UMM SALAL', 
            'UNAYZAH', 'ANTWERPEN EXPRESS', 'BASLE EXPRESS', 'ESSEN EXPRESS', 
            'DORTMUND EXPRESS', 'HONG KONG EXPRESS', 'LEVERKUSEN EXPRESS', 
            'LUDWIGSHAFEN EXPRESS', 'NEW YORK EXPRESS', 'SHANGHAI EXPRESS', 
            'ULSAN EXPRESS', 'RIO DE JANEIRO EXPRESS', 'MONTEVIDEO EXPRESS', 
            'BUENOS AIRES EXPRESS', 'VALPARAISO EXPRESS', 'CALLAO EXPRESS', 
            'CARTAGENA EXPRESS', 'GUAYAQUIL EXPRESS', 'SANTOS EXPRESS', 
            'HUMBOLDT EXPRESS', 'IQUIQUE EXPRESS', 'BREMEN EXPRESS', 
            'CHICAGO EXPRESS', 'COLOMBO EXPRESS', 'HANOVER EXPRESS', 
            'KUALA LUMPUR EXPRESS', 'KYOTO EXPRESS', 'OSAKA EXPRESS', 
            'TSINGTAO EXPRESS', 'FRANKFURT EXPRESS', 'PRAGUE EXPRESS', 
            'SOFIA EXPRESS', 'NAGOYA EXPRESS', 'BUDAPEST EXPRESS', 
            'VIENNA EXPRESS', 'CAUQUENES', 'CAUTIN', 'CISNES', 
            'COCHRANE', 'COPIAPO', 'CORCOVADO', 'COYHAIQUE', 'TEMPANOS', 
            'TENO', 'TIRUA', 'TOLTEN', 'TORRENTE', 'TUBUL', 'TUCAPEL', 
            'KIEL EXPRESS', 'DALIAN EXPRESS', 'NINGBO EXPRESS', 'YANTIAN EXPRESS', 
            'MAIPO', 'MEHUIN', 'KALAHARI EXPRESS', 'AL MANAMAH', 'AL SAFAT', 
            'JAZAN', 'MISSOURI EXPRESS', 'POTOMAC EXPRESS', 'DELAWARE EXPRESS', 
            'COLORADO EXPRESS', 'HUDSON EXPRESS', 'PALENA', 'SUAPE EXPRESS', 
            'LE HAVRE EXPRESS', 'DALLAS EXPRESS', 'AMSTERDAM EXPRESS', 
            'SEOUL EXPRESS', 'TOKYO EXPRESS', 'DUBLIN EXPRESS', 'GLASGOW EXPRESS', 
            'LIVERPOOL EXPRESS', 'DÜSSELDORF EXPRESS', 'KOBE EXPRESS', 
            'LONDON EXPRESS', 'MONTREAL EXPRESS', 'TORONTO EXPRESS', 
            'QUEBEC EXPRESS', 'ONTARIO EXPRESS', 'JAKARTA EXPRESS', 
            'ARICA EXPRESS', 'DACHAN BAY EXPRESS', 'MOMBASA EXPRESS', 
            'CHACABUCO', 'DUBAI EXPRESS', 'TEMA EXPRESS', 'LISBON EXPRESS', 
            'VALENCIA EXPRESS', 'SAN VICENTE EXPRESS', 'CABINDA EXPRESS', 
            'MATADI EXPRESS', 'DOUALA EXPRESS'   
]

In [ ]:
# Step 2: Create VesselListHL table with selected columns and filters
VesselListHL = last_positions.filter(col("SHIPNAME").isin(vessel_names)).select(
    "IMO",
    "SHIPNAME",
    "LAT",
    "LON",
    "DESTINATION",
    "TIMESTAMP",
    "AVG_SPEED",
    "COURSE",
    "CURRENT_PORT",
    "DISTANCE_TO_GO",
    "DISTANCE_TRAVELLED",
    "DRAUGHT",
    "DWT",
    "HEADING",
    "LAST_PORT",
    "MARKET",
    "MAIN_MARKET",
    "MAX_SPEED",
    "NEXT_PORT_NAME",
    "SPEED",
    "STATUS"
)

# Step 3: Define the LatestVesselOwnership query with a row number window function to get the latest record
vessel_ownership = session.table("VESSEL_OWNERSHIP")

# Window specification to partition by IMO and order by latest date
window_spec = Window.partition_by("IMO").order_by(coalesce(col("UPDATED_DATE"), col("CREATED_DATE")).desc())

# Define LatestVesselOwnership with row numbering for latest record selection
LatestVesselOwnership = vessel_ownership.select(
    "IMO",
    "COMPANY_TYPE",
    col("NAME").alias("COMPANY_NAME"),
    coalesce(col("UPDATED_DATE"), col("CREATED_DATE")).alias("LATEST_DATE"),
    row_number().over(window_spec).alias("rn")
).filter(col("rn") == 1)

In [ ]:
# Step 4: Join VesselListHL and LatestVesselOwnership to get the final result
query_result = VesselListHL.join(
    LatestVesselOwnership,
    VesselListHL["IMO"] == LatestVesselOwnership["IMO"],
    join_type="left"
).select(
    VesselListHL["IMO"],
    VesselListHL["SHIPNAME"],
    LatestVesselOwnership["COMPANY_TYPE"],
    LatestVesselOwnership["COMPANY_NAME"],
    VesselListHL["LAT"],
    VesselListHL["LON"],
    VesselListHL["AVG_SPEED"],
    VesselListHL["COURSE"],
    VesselListHL["CURRENT_PORT"],
    VesselListHL["DISTANCE_TO_GO"],
    VesselListHL["DISTANCE_TRAVELLED"],
    VesselListHL["DRAUGHT"],
    VesselListHL["DWT"],
    VesselListHL["HEADING"],
    VesselListHL["LAST_PORT"],
    VesselListHL["MARKET"],
    VesselListHL["MAIN_MARKET"],
    VesselListHL["MAX_SPEED"],
    VesselListHL["NEXT_PORT_NAME"],
    VesselListHL["SPEED"],
    VesselListHL["STATUS"]
)

# Convert result to Pandas DataFrame for easy viewing and manipulation
result_df = query_result.to_pandas()

# Display the result
print(result_df)